# Imports

In [ ]:
!pip install absl-py rouge-score nltk

In [ ]:
!python -m nltk.downloader punkt

In [ ]:
!pip install bert-score

In [1]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding
)
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import torch
import wandb
import evaluate  # Hugging Face's evaluate library
import numpy as np
import torch
from bert_score import BERTScorer, score as bert_score

from utils import tokenize_dataset_for_qna

/home/super_admin/hasindu/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configs

In [14]:
model_path = "../models/phi_qna_finetuned_attempt_3/final"

data_path = "../data/qna/"
test_data_path = data_path + "test.csv"

model_id = "microsoft/Phi-3.5-mini-instruct"
base_model_path = "../models/phi_pubmed_pretrained_attempt_3/final_pretrained"

max_len = 512
batch_size = 8

In [ ]:
wandb.init(project="qna_finetune-evaluation", name="attempt_3")

# Dataset

In [3]:
prompt_template = """
# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the question given below.

# Question: {question}

# Answer: """
prompt_template = prompt_template.strip()
print(prompt_template)

# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the question given below.

# Question: {question}

# Answer:


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [ ]:
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict


def tokenize_dataset_for_qna(tokenizer, data_df, prompt_template, max_len):
    dataset = Dataset.from_pandas(data_df)
    dataset = dataset.map(lambda sample: tokenize_for_qna(sample, tokenizer, prompt_template, max_len), batched=False)
    return dataset


def tokenize_for_qna(example, tokenizer, prompt_template, max_len):
    prompt = prompt_template.format(question=example['question'])
    answer = example["answer"] + tokenizer.eos_token
    full_text = prompt + answer

    # Tokenize answer to get the length of answer tokens
    prompt_len = len(tokenizer(
        prompt,
        truncation=True,
        max_length=max_len
    )["input_ids"])

    # Tokenize answer to get the length of answer tokens
    full_len = len(tokenizer(
        full_text,
        truncation=False
    )["input_ids"])
    
    # Tokenize full sequence once
    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=max_len,
        padding="max_length",
        return_attention_mask=True    
    )
    
    # Convert to numpy arrays for faster operations
    input_ids = np.array(tokenized["input_ids"])
    attention_mask = np.array(tokenized["attention_mask"])
    
    # Create labels array and mask prompt portion efficiently
    labels = input_ids.copy()
    padding_len = max_len - full_len
    # Mask the prompt tokens
    labels[padding_len:padding_len + prompt_len] = -100
    
    # Update the tokenized dict with numpy arrays
    tokenized["input_ids"] = input_ids.tolist()
    tokenized["attention_mask"] = attention_mask.tolist()
    tokenized["labels"] = labels.tolist()
    
    return tokenized


In [ ]:
test_df = pd.read_csv(test_data_path)
test_set = tokenize_dataset_for_qna(tokenizer, test_df.iloc[:1500, :], prompt_template, max_len)

In [ ]:
test_set[0]

# Model

In [ ]:
peft_config = PeftConfig.from_pretrained(model_path)
base_model_name = peft_config.base_model_name_or_path

In [ ]:
base_model_name

In [5]:
# === Quantized model loading ===
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    # quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.97it/s]


In [6]:
model = PeftModel.from_pretrained(model, model_path)
model.eval() 

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=24, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=24, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vect

### Base model

In [16]:
model_id

'microsoft/Phi-3.5-mini-instruct'

In [17]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)
model.eval()

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.95it/s]


Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./eval_output_base",
#     per_device_eval_batch_size=batch_size,
#     do_eval=True,
#     report_to="none"
# )

# base_model_trainer = Trainer(
#     model=base_model,
#     args=training_args,
#     tokenizer=tokenizer,
#     data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
# )

# base_model_eval_result = base_model_trainer.evaluate(test_set)

# Test set evaluation

In [ ]:
training_args = TrainingArguments(
    output_dir="./eval_output",
    per_device_eval_batch_size=batch_size,
    do_eval=True,
    report_to="none",
    eval_accumulation_steps=2,
)

In [ ]:

# Initialize metrics ONCE (reuse them)
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
bert_scorer = BERTScorer(
    lang="en",
    model_type="bert-base-uncased",
    device="cuda" if torch.cuda.is_available() else "cpu",
    idf=False,  # Disable IDF to save memory
    rescale_with_baseline=True  # Better score normalization
)

def compute_metrics(eval_preds):
    """ Metric computation """
    with torch.no_grad():
        logits, labels = eval_preds
        
        # Convert to numpy (move to CPU first if needed)
        if torch.is_tensor(logits):
            logits = logits.detach().cpu().numpy()
        if torch.is_tensor(labels):
            labels = labels.detach().cpu().numpy()
        
        # Get predicted tokens (shape: [batch_size, seq_length])
        pred_ids = np.argmax(logits, axis=-1)
        
        # Decode in batches to avoid memory spikes
        batch_size = 8  # Adjust based on your GPU memory
        pred_str, label_str = [], []
        
        for i in range(0, len(pred_ids), batch_size):
            # Decode predictions
            batch_preds = pred_ids[i:i+batch_size]
            pred_str.extend(tokenizer.batch_decode(
                batch_preds, 
                skip_special_tokens=True
            ))
            
            # Decode labels (replace -100 with pad_token_id)
            batch_labels = labels[i:i+batch_size]
            batch_labels = np.where(
                batch_labels != -100, 
                batch_labels, 
                tokenizer.pad_token_id
            )
            label_str.extend(tokenizer.batch_decode(
                batch_labels, 
                skip_special_tokens=True
            ))
        
        # Skip if empty (avoid errors)
        if not pred_str or not label_str:
            return {
                'bleu': 0.0,
                'rouge1': 0.0,
                'rouge2': 0.0,
                'rougeL': 0.0,
                'bertscore_f1': 0.0
            }
        
        # Compute BLEU (handle edge cases)
        try:
            bleu_score = bleu_metric.compute(
                predictions=pred_str,
                references=[[ref] for ref in label_str]
            )['bleu']
        except:
            bleu_score = 0.0
        
        # Compute ROUGE
        rouge_scores = rouge_metric.compute(
            predictions=pred_str,
            references=label_str,
            use_stemmer=True
        )
        
        # Compute BERTScore in batches
        P, R, F1 = bert_scorer.score(
            pred_str, 
            label_str,
            batch_size=4  # Small batch for BERTScore
        )
        
        metrics = {
            'bleu': bleu_score,
            'rouge1': rouge_scores['rouge1'],
            'rouge2': rouge_scores['rouge2'],
            'rougeL': rouge_scores['rougeL'],
            'bertscore_precision': P.mean().item(),
            'bertscore_recall': R.mean().item(),
            'bertscore_f1': F1.mean().item(),
        }
        
        torch.cuda.empty_cache()
        return metrics

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer, padding=False),
    compute_metrics=compute_metrics
)

In [ ]:
# === Evaluate perplexity ===
eval_result = trainer.evaluate(test_set)

In [ ]:
# Print all results
print("\nEvaluation Metrics:")
print(f"Loss: {eval_result['eval_loss']:.4f}")
print(f"Perplexity: {torch.exp(torch.tensor(eval_result['eval_loss'])):.2f}")
print(f"BLEU: {eval_result['eval_bleu']:.4f}")
print(f"ROUGE-1: {eval_result['eval_rouge1']:.4f}")
print(f"ROUGE-2: {eval_result['eval_rouge2']:.4f}")
print(f"ROUGE-L: {eval_result['eval_rougeL']:.4f}")
print(f"BERTscore precision: {eval_result['eval_bertscore_precision']:.4f}")
print(f"BERTscore recall: {eval_result['eval_bertscore_recall']:.4f}")
print(f"BERTscore f1: {eval_result['eval_bertscore_f1']:.4f}")

In [ ]:
wandb.log({
    "eval_loss": eval_result['eval_loss'], 
    "perplexity": torch.exp(torch.tensor(eval_result['eval_loss'])),
    "BLUE": eval_result['eval_bleu'],
    "ROUGE_1": eval_result['eval_rouge1'],
    "ROUGE_2": eval_result['eval_rouge2'],
    "ROUGE_L": eval_result['eval_rougeL'],
    "BERTscore_precision": eval_result['eval_bertscore_precision'],
    "BERTscore recall": eval_result['eval_bertscore_recall'],
    "BERTscore f1": eval_result['eval_bertscore_f1']
})

# Inference

In [ ]:
samples = test_set.select(range(433, 439))  # First 5 examples
input_ids = torch.tensor(samples["input_ids"]).to(model.device)
attention_mask = torch.tensor(samples["attention_mask"]).to(model.device)

generated_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=128,
    do_sample=False,
    use_cache=False
)

generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

# Log predictions to W&B
wandb_table = wandb.Table(columns=["Title", "Actual Abstract", "Generated Text"])
for i, gen in enumerate(generated_texts):
    title = samples[i]["title"]
    actual = samples[i]["abstract"]
    print(f"\nActual: {title}\n{actual}\n---\nGenerated: {gen}\n")
    wandb_table.add_data(title, actual, gen)




In [ ]:
samples[0]

In [ ]:
wandb.log({"generated_examples": wandb_table})

In [ ]:
def generate(model, text, max_new_tokens=128):
    sample = tokenizer(text + tokenizer.eos_token, truncation=True, padding="max_length", max_length=max_len, return_attention_mask=True)
    input_ids = torch.tensor([sample["input_ids"]]).to(model.device)
    attention_mask = torch.tensor([sample["attention_mask"]]).to(model.device)
    
    generated_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_beams=1,
        do_sample=False,
        use_cache=False
    )
    
    generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    return generated_texts[0]
    # # Log predictions to W&B
    # for i, gen in enumerate(generated_texts):
    #     title = samples[i]["title"]
    #     actual = samples[i]["abstract"]
    #     print(f"\nTitle: {title}\n---\nActual Abstract: {actual}\n---\nGenerated: {gen}\n")
    #     wandb_table.add_data(title, actual, gen)
    
    
    # wandb.log({"generated_examples": wandb_table})

In [ ]:
generated_text = generate(model, prompt_template.format(question="What are the symptoms of Glaucoma ??"), max_new_tokens=200)	
print(generated_text)

# Instruction:
Assume you are an excellent doctor. Using your knowledge, answer the quesion given below.

# Question: What are the symptoms of Glaucoma ??

# Answer:ains of Glaucoma can vary depending on the type and severity of the condition. The most common symptom of glaucoma is loss of vision, which may begin with a loss of peripheral (side) vision. This is often described as tunnel vision.  Glaucoma can also cause blurred vision, halos around lights, eye pain, redness, and vision loss.  Glaucoma is often called the "silent thief of sight" because it usually has no symptoms until significant vision loss has occurred.  If you have glaucoma, you may not notice any changes in your vision until the damage is severe.  Glaucoma is a progressive disease, which means it gets worse over time.  If you have glaucoma, it's important to have regular eye exams to monitor your vision and eye pressure.  If you have any of the following symptoms, you should see


In [ ]:
wandb.log({"example_2": generated_text})

In [ ]:
test_set[0]

In [ ]:
generate(base_model, "# The relationship between diabetes and blood pressure\n")

In [11]:
def stream_generate(model, tokenizer, text, max_new_tokens=128):
    model.eval()
    sample = tokenizer(
        text + tokenizer.eos_token,
        return_tensors="pt",
        truncation=True,
        max_length=max_len
    ).to(model.device)

    input_ids = sample["input_ids"]
    generated = input_ids.clone()
    past_key_values = None
    position_ids = torch.arange(0, input_ids.shape[1], device=model.device).unsqueeze(0)

    prev_decoded = tokenizer.decode(generated[0], skip_special_tokens=True)

    for i in range(max_new_tokens):
        if i == 0:
            input_token = input_ids
        else:
            input_token = next_token_id
            position_ids = torch.tensor([[generated.shape[1] - 1]], device=model.device)

        with torch.no_grad():
            outputs = model(
                input_ids=input_token,
                past_key_values=past_key_values,
                use_cache=True,
                position_ids=position_ids
            )

        logits = outputs.logits[:, -1, :]
        next_token_id = torch.argmax(logits, dim=-1, keepdim=True)

        generated = torch.cat((generated, next_token_id), dim=1)
        past_key_values = outputs.past_key_values

        # Decode full sequence and compute the diff
        decoded = tokenizer.decode(generated[0], skip_special_tokens=True)
        new_text = decoded[len(prev_decoded):]
        prev_decoded = decoded

        yield new_text

        if next_token_id.squeeze().item() == tokenizer.eos_token_id:
            break


In [19]:
for token in stream_generate(model, tokenizer, prompt_template.format(question="What are the symptoms of Glaucoma ??")):
    print(token, end='', flush=True)

 What is the value of x in the equation 3x + 5 = 14? Please show your work.

# Answer
To find the value of x in the equation 3x + 5 = 14, follow these steps:

1. Subtract 5 from both sides of the equation to isolate the term with x on one side:
   3x + 5 - 5 = 14 - 5
   3x = 9

2. Divide both sides of the equation by 3 to solve for x:
   3x /